In [1]:
import numpy as np
import tensorflow as tf

# Len(params)=1
就是看params是由几个tensor组成.

## Lookup a value from a vector

In [21]:
sess=tf.InteractiveSession()

/home/chet/.local/share/virtualenvs/Python_Code-ckxiZQVL/lib/python3.6/site-packages/tensorflow/python/client/session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [3]:
params = tf.constant([10,20,30,40])
ids = tf.constant([0,1,2,3])

print (tf.nn.embedding_lookup(params,ids).eval())

[10 20 30 40]


## Lookup from a matrix
index表示parameters的哪一行.

In [27]:
params = tf.random_uniform(shape=[4,3],minval=0, maxval=50,dtype=tf.int32,seed=2)
ids = tf.constant([1,2])
# ids_2 = tf.constant([1,2])
lookup = tf.nn.embedding_lookup(params, ids)
# lookup_2 = tf.nn.embedding_lookup(params, ids_2)

Parameters, IDs, Lookup = sess.run([params, ids, lookup])
print('Parameter Matrix is:\n', Parameters)
print('\nIDs is: \n', sess.run(ids))
print('\nEmbedding Lookup Result is:\n', Lookup)

Parameter Matrix is:
 [[41  6 46]
 [27 39 45]
 [47 27 24]
 [26 39 13]]

IDs is: 
 [1 2]

Embedding Lookup Result is:
 [[27 39 45]
 [47 27 24]]


# Len(params)>1
**总结**：就是params总数个元素平均分到len(params)个partition中，#params/len(params)不能整除的，就把尾部的partition中不够数的空着。然后ids的对应法则详见partition strategy.(这个理解之后其实还是很简单的...)
- ids不再直接表示parameters的哪一行, 而是通过Partition Strategy, 针对不同的tensor简介索引对应row.
    - 所以写ids的时候就要更具规则好好写. 不然你自己都不知道索引到的是什么.
- If len(params) > 1, each element id of ids is partitioned between the elements of params according to the partition_strategy.
- In all strategies, if the id space does not evenly divide the number of partitions, each of the first (max_id + 1) % len(params) partitions will be assigned one more id.
## Partition Strategy
    - partition_strategy controls the way how the ids are distributed among the list. The partitioning is useful for larger scale problems when the matrix might be too large to keep in one piece.

### mod
- **p = id % len(params) **

In [16]:
ids=[0, 3, 6, 1, 2, 5, 8]
[a%3 for a in ids]


# Therefore Partition would be:
# [[0, 0, 0], 
#  [1], 
#  [2, 2, 2]]

# ie. 
# [[0, 3, 6], 
#  [1], 
# [2, 5, 8]]

[0, 0, 0, 1, 2, 2, 2]

In [18]:
'''Partition Strategy: mod'''

def test_embedding_lookup():
    a = np.arange(12).reshape(3, 4)
    b = np.arange(12, 16).reshape(1, 4)
    c = np.arange(16, 28).reshape(3, 4)
    print('a:\n',a)
    print('\n')
    print('b:\n',b)
    print('\n')
    print('c:\n',c)
    print('\n')

    a = tf.Variable(a)
    b = tf.Variable(b)
    c = tf.Variable(c)

    t = tf.nn.embedding_lookup([a, b, c],
       partition_strategy='mod', ids=[0, 3, 6, 1, 2, 5, 8])

    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    m = sess.run(t)
    print('params:',sess.run([a, b, c]))
    print('\nlen(params)=',len([a,b,c]))
    print('\nids=[0, 3, 6, 1, 2, 5, 8]\n')
    print('embedding_lookup:\n',m)


test_embedding_lookup()

# p = id % len(params)

# 分析:
# ====================================================================================================
#  这里我们注意到params是由[a, b, c]这三个tensor组成。即len(params)=3,且a,b,c这三个tensor的第一维度分别为3,1,3。
#  在把这个三个tensor组合过程中，我们按照partition_strategy='mod'策略分割。即每个tensor的元素之间相差len(params).
#  这里分割方式为[a, b, c]  == [[0,3,6], [1,4,7], [2,5,8]]  
# ====================================================================================================
#  这里程序还不知道4和7是找不到对应的元素的，在获取元素时候会报错
#     a=[[ 0  1  2  3]     = [0, 3, 6]  -->  [0  1  2  3]  = 0
#        [ 4  5  6  7]                  -->  [4  5  6  7]  = 3
#        [ 8  9 10 11]]                 -->  [8  9 10 11]  = 6

#     b=[[12 13 14 15]]    = [1, 4, 7]  -->  [12 13 14 15] = 1
#                                       -->  运行时报错  = 4
#                                       -->  运行时报错  = 7
#                            1,4,7这几个ids都是被assign给b的，但是params里没有对应的，所以绝对不能call id:4,7.

#     c =[[16 17 18 19]    = [2, 5, 8]  -->  ...
#         [20 21 22 23]
#         [24 25 26 27]]

# 输出:
#     [[ 0  1  2  3]
#      [ 4  5  6  7]
#      [ 8  9 10 11]]

#     [[12 13 14 15]]

#     [[16 17 18 19]
#      [20 21 22 23]
#      [24 25 26 27]]

#     [[ 0  1  2  3]  # 0
#      [ 4  5  6  7]  # 3
#      [ 8  9 10 11]  # 6
#      [12 13 14 15]  # 1
#      [16 17 18 19]  # 2
#      [20 21 22 23]  # 5
#      [24 25 26 27]] # 8



a:
 [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


b:
 [[12 13 14 15]]


c:
 [[16 17 18 19]
 [20 21 22 23]
 [24 25 26 27]]


params: [array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]]), array([[12, 13, 14, 15]]), array([[16, 17, 18, 19],
       [20, 21, 22, 23],
       [24, 25, 26, 27]])]

len(params)= 3

ids=[0, 3, 6, 1, 2, 5, 8]

embedding_lookup:
 [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]
 [16 17 18 19]
 [20 21 22 23]
 [24 25 26 27]]


In [17]:
'''Partition Strategy: mod'''

def test_embedding_lookup():
    a = np.arange(12).reshape(3, 4)
    b = np.arange(12, 16).reshape(1, 4)
    c = np.arange(16, 28).reshape(3, 4)
    print('a:\n',a)
    print('\n')
    print('b:\n',b)
    print('\n')
    print('c:\n',c)
    print('\n')

    a = tf.Variable(a)
    b = tf.Variable(b)
    c = tf.Variable(c)

    t = tf.nn.embedding_lookup([a, b, c],
       partition_strategy='mod', ids=[0, 6, 1, 2, 5, 8])

    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    m = sess.run(t)
    print('params:',sess.run([a, b, c]))
    print('\nlen(params)=',len([a,b,c]))
    print('\nids=[0, 6, 1, 2, 5, 8]\n')
    print('embedding_lookup:\n',m)


test_embedding_lookup()

# p = id % len(params)

# 分析:
# ====================================================================================================
#  这里我们注意到params是由[a, b, c]这三个tensor组成。即len(params)=3,且a,b,c这三个tensor的第一维度分别为3,1,3。
#  在把这个三个tensor组合过程中，我们按照partition_strategy='mod'策略分割。即每个tensor的元素之间相差len(params).
#  这里分割方式为[a, b, c]  == [[0,3,6], [1,4,7], [2,5,8]]  
# ====================================================================================================
#  这里程序还不知道4和7是找不到对应的元素的，在获取元素时候会报错
#     a=[[ 0  1  2  3]     = [0, 3, 6]  -->  [0  1  2  3]  = 0
#        [ 4  5  6  7]                  -->  [4  5  6  7]  = 3 (missing)
#        [ 8  9 10 11]]                 -->  [8  9 10 11]  = 6

#     b=[[12 13 14 15]]    = [1, 4, 7]  -->  [12 13 14 15] = 1
#                                       -->  运行时报错  = 4
#                                       -->  运行时报错  = 7
#                            1,4,7这几个ids都是被assign给b的，但是params里没有对应的，所以绝对不能call id:4,7.

#     c =[[16 17 18 19]    = [2, 5, 8]  -->  ...
#         [20 21 22 23]
#         [24 25 26 27]]

# 输出:
#     [[ 0  1  2  3]
#      [ 4  5  6  7]
#      [ 8  9 10 11]]

#     [[12 13 14 15]]

#     [[16 17 18 19]
#      [20 21 22 23]
#      [24 25 26 27]]

#     [[ 0  1  2  3]  # 0
#      [ 8  9 10 11]  # 6

#      [12 13 14 15]  # 1

#      [16 17 18 19]  # 2
#      [20 21 22 23]  # 5
#      [24 25 26 27]] # 8



a:
 [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


b:
 [[12 13 14 15]]


c:
 [[16 17 18 19]
 [20 21 22 23]
 [24 25 26 27]]


params: [array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]]), array([[12, 13, 14, 15]]), array([[16, 17, 18, 19],
       [20, 21, 22, 23],
       [24, 25, 26, 27]])]

len(params)= 3

ids=[0, 6, 1, 2, 5, 8]

embedding_lookup:
 [[ 0  1  2  3]
 [ 8  9 10 11]
 [12 13 14 15]
 [16 17 18 19]
 [20 21 22 23]
 [24 25 26 27]]


In [19]:
ids=[0, 3, 6, 1, 4, 7, 2]
[a%3 for a in ids]

[0, 0, 0, 1, 1, 1, 2]

In [20]:
 def test_embedding_lookup():
    a = np.arange(12).reshape(3, 4)
    b = np.arange(12, 16).reshape(1, 4)
    c = np.arange(16, 28).reshape(3, 4)
    print(a)
    print('\n')
    print(b)
    print('\n')
    print(c)
    print('\n')
    
    a = tf.Variable(a)
    b = tf.Variable(b)
    c = tf.Variable(c)

    t = tf.nn.embedding_lookup([a, c, b],
       partition_strategy='mod', ids=[0, 3, 6, 1, 4, 7, 2]) # 注意啊，上面的[a,c,b]

    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    m = sess.run(t)
    print(m)


test_embedding_lookup()


# 分析:
# ====================================================================================================
#   这里我们把params从[a, b, c]改为[a, c, b]这三个tensor组成。a,c,b这三个tensor的第一维度分别为3,3,1。
#   在把这个三个tensor组合过程中，依旧是每个tensor的元素之间相差len(params).这里分割方式为[a, c, b]  == [[0,3,6], [1,4,7], [2,5,8]]  
#   这里程序还不知道4和7是找不到对应的元素的，在获取元素时候会报错
# ====================================================================================================

#     a=[[ 0  1  2  3]     = [0, 3, 6]  -->  [0  1  2  3]  = 0
#        [ 4  5  6  7]                  -->  [4  5  6  7]  = 3
#        [ 8  9 10 11]]                 -->  [8  9 10 11]  = 6

#     c=[[16 17 18 19]     = [1, 4, 7]  -->  [16 17 18 19]  = 1
#        [20 21 22 23]                  -->  [20 21 22 23]  = 4
#        [24 25 26 27]]                -->  [24 25 26 27]  = 7


#     b=[[12 13 14 15]]    = [2, 5, 8]  -->  [12 13 14 15] = 2
#                                       -->  运行时报错  = 5 (并不会报错)
#                                       -->  运行时报错  = 8      


#     输出:
#     [[ 0  1  2  3]
#      [ 4  5  6  7]
#      [ 8  9 10 11]]

#     [[12 13 14 15]]

#     [[16 17 18 19]
#      [20 21 22 23]
#      [24 25 26 27]]

#     [[ 0  1  2  3]  # 0
#      [ 4  5  6  7]  # 3
#      [ 8  9 10 11]  # 6
#      [16 17 18 19]  # 1
#      [20 21 22 23]  # 4
#      [24 25 26 27]  # 7
#      [12 13 14 15]] # 2

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


[[12 13 14 15]]


[[16 17 18 19]
 [20 21 22 23]
 [24 25 26 27]]


[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [16 17 18 19]
 [20 21 22 23]
 [24 25 26 27]
 [12 13 14 15]]


### div
- 就是按照顺序排列嘛...
- If partition_strategy is "div", we assign ids to partitions in a contiguous manner. In this case, 13 ids are split across 5 partitions as: [[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10], [11, 12]]

In [66]:
def test_embedding_lookup():
    a = np.arange(12).reshape(3, 4)
    b = np.arange(12, 16).reshape(1, 4)
    c = np.arange(16, 28).reshape(3, 4)
    print('a:\n',a)
    print('\n')
    print('b:\n',b)
    print('\n')
    print('c:\n',c)
    print('\n')

    a = tf.Variable(a)
    b = tf.Variable(b)
    c = tf.Variable(c)

    t = tf.nn.embedding_lookup([a, b, c],
       partition_strategy='div', ids=[0, 1, 2, 3, 5, 6])

    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    m = sess.run(t)
    print(m)


test_embedding_lookup()


# 分析:
# ====================================================================================================
#     这里我们把params依旧是[a, b, c]，三个tensor的第一维度分别为3,1,3。
#     在把这个三个tensor组合过程中，这我们按照partition_strategy='div'策略分割。即每个tensor的元素之间相差1.
#     如果不够等分的话，前面(max_id+1)%len(params)多分一个元素。这里一共7个元素，分为3组，即3、2、2分配。
#     这边的元素表示三个tensor的第一维度之和。
#     这里分割方式为[a, b, c]  == [[0,1,2], [3,4], [5,6]]  
# ====================================================================================================

#     这里程序还不知道4和7是找不到对应的元素的，在获取元素时候会报错
#     a=[[ 0  1  2  3]     = [0, 1, 2]  -->  [0  1  2  3]  = 0
#        [ 4  5  6  7]                  -->  [4  5  6  7]  = 1
#        [ 8  9 10 11]]                 -->  [8  9 10 11]  = 2




#     b=[[12 13 14 15]]    = [3, 4]  -->  [12 13 14 15] = 3
#                                       -->  运行时报错  = 4

#     c=[[16 17 18 19]     = [5, 6]  -->  [16 17 18 19]  = 5
#        [20 21 22 23]                  -->  [20 21 22 23]  = 6
#        [24 25 26 27]]                -->  [24 25 26 27]  = 这个是找不到的了   


#     输出:
#     [[ 0  1  2  3]
#      [ 4  5  6  7]
#      [ 8  9 10 11]]

#     [[12 13 14 15]]

#     [[16 17 18 19]
#      [20 21 22 23]
#      [24 25 26 27]]

#     [[ 0  1  2  3]  # 0
#      [ 4  5  6  7]  # 1
#      [ 8  9 10 11]  # 2
#      [12 13 14 15]  # 3
#      [16 17 18 19]  # 5
#      [20 21 22 23]] # 6


a:
 [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


b:
 [[12 13 14 15]]


c:
 [[16 17 18 19]
 [20 21 22 23]
 [24 25 26 27]]


[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]
 [16 17 18 19]
 [20 21 22 23]]


In [43]:
def test_embedding_lookup():
    a = np.arange(12).reshape(3, 4)
    b = np.arange(12, 16).reshape(1, 4)
    c = np.arange(16, 28).reshape(3, 4)
    print(a)
    print('\n')
    print(b)
    print('\n')
    print(c)
    print('\n')

    a = tf.Variable(a)
    b = tf.Variable(b)
    c = tf.Variable(c)

    t = tf.nn.embedding_lookup([a, c, b],
       partition_strategy='div', ids=[0, 1, 2, 3, 4, 5])

    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    m = sess.run(t)
    print(m)


test_embedding_lookup()

# 分析:
#     这里我们把params改为[a, c, b]，三个tensor的第一维度分别为3,3,1。

#     在把这个三个tensor组合过程中，这我们按照partition_strategy='div'策略分割。这里一共7个元素，分为3组，即3、2、2分配。

#     这里分割方式为[a, c, b]  == [[0,1,2], [3,4], [5,6]]  

#     这里程序还不知道4和7是找不到对应的元素的，在获取元素时候会报错
#     a=[[ 0  1  2  3]     = [0, 1, 2]  -->  [0  1  2  3]  = 0
#        [ 4  5  6  7]                  -->  [4  5  6  7]  = 1
#        [ 8  9 10 11]]                 -->  [8  9 10 11]  = 2

#     c=[[16 17 18 19]     = [3, 4]  -->  [16 17 18 19]  = 3
#        [20 21 22 23]                  -->  [20 21 22 23]  = 4
#        [24 25 26 27]]                -->  [24 25 26 27]  = 这个是找不到的了   

#     b=[[12 13 14 15]]    = [5, 6]  -->  [12 13 14 15] = 5
#                                       -->  运行时报错  = 6


#     输出:
#     [[ 0  1  2  3]
#      [ 4  5  6  7]
#      [ 8  9 10 11]]

#     [[12 13 14 15]]

#     [[16 17 18 19]
#      [20 21 22 23]
#      [24 25 26 27]]

#     [[ 0  1  2  3]  # 0
#      [ 4  5  6  7]  # 1
#      [ 8  9 10 11]  # 2
#      [16 17 18 19]  # 3
#      [20 21 22 23]  # 4
#      [16 17 18 19]] # 5


[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


[[12 13 14 15]]


[[16 17 18 19]
 [20 21 22 23]
 [24 25 26 27]]


[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [16 17 18 19]
 [20 21 22 23]
 [12 13 14 15]]


In [73]:
def test_embedding_lookup():
    '''
    这个是对的，可以这么数和index，不过id要跳过b没有的部分
    '''
    a = np.arange(12).reshape(3, 4)
    b = np.arange(12, 16).reshape(1, 4)
    c = np.arange(16, 28).reshape(3, 4)
    print('a:\n',a)
    print('\n')
    print('b:\n',b)
    print('\n')
    print('c:\n',c)
    print('\n')

    a = tf.Variable(a)
    b = tf.Variable(b)
    c = tf.Variable(c)

    t = tf.nn.embedding_lookup([a, b, c],
       partition_strategy='div', ids=[0, 1, 2, 3, 5, 6])

    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    m = sess.run(t)
    print('ids=[0, 1, 2, 3, 5, 6]\n')
    print(m)


test_embedding_lookup()

# 分析:
#     这里我们把params改为[a, c, b]，三个tensor的第一维度分别为3,3,1。

#     在把这个三个tensor组合过程中，这我们按照partition_strategy='div'策略分割。这里一共7个元素，分为3组，即3、2、2分配。

#     这里分割方式为[a, c, b]  == [[0,1,2], [3,4], [5,6]]  

#     这里程序还不知道4和7是找不到对应的元素的，在获取元素时候会报错
#     a=[[ 0  1  2  3]     = [0, 1, 2]  -->  [0  1  2  3]  = 0
#        [ 4  5  6  7]                  -->  [4  5  6  7]  = 1
#        [ 8  9 10 11]]                 -->  [8  9 10 11]  = 2

#     b=[[12 13 14 15]]    = [3, 4]  -->  [12 13 14 15] = 3
#                                       -->  运行时报错  = 4

#     c=[[16 17 18 19]     = [5, 6]  -->  [16 17 18 19]  = 5
#        [20 21 22 23]                  -->  [20 21 22 23]  = 6
#        [24 25 26 27]]                -->  [24 25 26 27]  = 这个是找不到的了   




#     输出:
#     [[ 0  1  2  3]
#      [ 4  5  6  7]
#      [ 8  9 10 11]]

#     [[12 13 14 15]]

#     [[16 17 18 19]
#      [20 21 22 23]
#      [24 25 26 27]]

#     [[ 0  1  2  3]  # 0
#      [ 4  5  6  7]  # 1
#      [ 8  9 10 11]  # 2
#      [16 17 18 19]  # 3
#      [20 21 22 23]  # 4
#      [16 17 18 19]] # 5


a:
 [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


b:
 [[12 13 14 15]]


c:
 [[16 17 18 19]
 [20 21 22 23]
 [24 25 26 27]]


ids=[0, 1, 2, 3, 5, 6]

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]
 [16 17 18 19]
 [20 21 22 23]]


In [75]:
def test_embedding_lookup():
    '''
    这个是错的，ids太长了，7数不到，比元素数长了
    '''
    a = np.arange(12).reshape(3, 4)
    b = np.arange(12, 16).reshape(1, 4)
    c = np.arange(16, 28).reshape(3, 4)
    print('a:\n',a)
    print('\n')
    print('b:\n',b)
    print('\n')
    print('c:\n',c)
    print('\n')

    a = tf.Variable(a)
    b = tf.Variable(b)
    c = tf.Variable(c)

    t = tf.nn.embedding_lookup([a, b, c],
       partition_strategy='div', ids=[0, 1, 2, 3, 5, 6, 7])

    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    print('ids=[0, 1, 2, 3, 5, 6, 7]\n')
    m = sess.run(t)
    print(m)


test_embedding_lookup()

# 分析:
#     这里我们把params改为[a, c, b]，三个tensor的第一维度分别为3,3,1。

#     在把这个三个tensor组合过程中，这我们按照partition_strategy='div'策略分割。这里一共7个元素，分为3组，即3、2、2分配。

#     这里分割方式为[a, c, b]  == [[0,1,2], [3,4], [5,6]]  

#     这里程序还不知道4和7是找不到对应的元素的，在获取元素时候会报错
#     a=[[ 0  1  2  3]     = [0, 1, 2]  -->  [0  1  2  3]  = 0
#        [ 4  5  6  7]                  -->  [4  5  6  7]  = 1
#        [ 8  9 10 11]]                 -->  [8  9 10 11]  = 2

#     b=[[12 13 14 15]]    = [3, 4]  -->  [12 13 14 15] = 3
#                                       -->  运行时报错  = 4

#     c=[[16 17 18 19]     = [5, 6]  -->  [16 17 18 19]  = 5
#        [20 21 22 23]                  -->  [20 21 22 23]  = 6
#        [24 25 26 27]]                -->  [24 25 26 27]  = 这个是找不到的了   




#     输出:
#     [[ 0  1  2  3]
#      [ 4  5  6  7]
#      [ 8  9 10 11]]

#     [[12 13 14 15]]

#     [[16 17 18 19]
#      [20 21 22 23]
#      [24 25 26 27]]

#     [[ 0  1  2  3]  # 0
#      [ 4  5  6  7]  # 1
#      [ 8  9 10 11]  # 2
#      [16 17 18 19]  # 3
#      [20 21 22 23]  # 4
#      [16 17 18 19]] # 5


a:
 [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


b:
 [[12 13 14 15]]


c:
 [[16 17 18 19]
 [20 21 22 23]
 [24 25 26 27]]


ids=[0, 1, 2, 3, 5, 6, 7]



InvalidArgumentError: partitions[6] = 3 is not in [0, 3)
	 [[Node: embedding_lookup_42/DynamicPartition_1 = DynamicPartition[T=DT_INT32, num_partitions=3, _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_lookup_42/range, embedding_lookup_42/Maximum)]]

Caused by op 'embedding_lookup_42/DynamicPartition_1', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/usr/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 127, in start
    self.asyncio_loop.run_forever()
  File